In [ ]:
!modelscope download --dataset liucong/Chinese-DeepSeek-R1-Distill-data-110k distill_r1_110k.jsonl --local_dir /home/lixinyu/data/Chinese-DeepSeek-R1-Distill-data-110k
!modelscope download --dataset AI-ModelScope/alpaca-gpt4-data-zh --local_dir /home/lixinyu/data/alpaca-gpt4-data-zh

In [35]:
from modelscope.msdatasets import MsDataset
ds =  MsDataset.load('/home/lixinyu/data/alpaca-gpt4-data-zh', subset_name='default', split='train')
print(ds)

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-25 18:53:19,526 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from /home/lixinyu/data/alpaca-gpt4-data-zh. Please make sure that you can trust the external codes.
2025-07-25 18:53:19,529 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from /home/lixinyu/data/alpaca-gpt4-data-zh. Please make sure that you can trust the external codes.
Generating train split: 48818 examples [00:00, 57722.68 examples/s]

In [47]:
data=[]
for ds_data in ds:
    data.append(ds_data)

In [48]:
import random

# 假设 data 已经通过前面代码加载
sample = random.sample(data, 10)   # 随机选 10 条
print(sample[1])

{'instruction': '用以下音符写一个500字的故事：\nC大调、D小调、F大调、G大调。', 'input': None, 'output': '从前，有一只小鸟，它生活在一个叫C大调的森林里。这里的空气清新，阳光灿烂，小河潺潺流过，各种颜色的花朵在阳光下绽放，小鸟每天都快快乐乐地唱着歌，欢快的旋律在森林中回荡。\n\n有一天，小鸟决定离开家乡，去周围的森林探险。它飞过了一座山，来到了一个名叫D小调的森林。这里的天空灰蒙蒙的，枯树瘦影，小鸟不禁有些害怕。它唱起了歌，声音却是低沉而忧伤的。\n\n小鸟觉得这里太过阴郁，便继续前行。不久后，它来到了F大调森林。这里绿草茵茵，鲜花盛开，阳光洒在鸟身上，小鸟感到无比温暖。它欢快地唱着歌，腾空飞起，自由自在。\n\n终于，小鸟来到了它旅程的终点，一个名叫G大调的森林。这里有着壮丽的瀑布，清澈的湖泊和高耸的树木。小鸟沐浴在阳光下，感到一种前所未有的喜悦。它高声歌唱，声音清脆悦耳。\n\n小鸟的旅程就这样结束了，它带着满满'}


In [49]:
import json

# 将 sample（10条数据）写入新的 jsonl 文件
with open('./data/sample.jsonl', 'w', encoding='utf-8') as f:
    for item in sample:
        # 再次确认没有 None
        if item["input"] is None:
            item["input"] = ""
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

In [51]:
prompt=sample[0]["instruction"]+"\n"+sample[0]["input"]
prompt

'告诉我一个改善你生活方式生态足迹的方法。\n'

In [ ]:
import os
from openai import OpenAI


client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen3-235b-a22b-thinking-2507",
    messages=[
        {"role": "user", "content": prompt},
    ],
    # Qwen3模型通过enable_thinking参数控制思考过程（开源版默认True，商业版默认False）
    # 使用Qwen3开源版模型时，若未启用流式输出，请将下行取消注释，否则会报错
    # extra_body={"enable_thinking": False},
    max_tokens=512,
    extra_body={"enable_thinking": True,"thinking_budget":1024}
)


In [61]:
# 通过reasoning_content字段打印思考过程
print("思考过程：")
print(completion.choices[0].message.reasoning_content)

# 通过content字段打印最终答案
print("最终答案：")
print(completion.choices[0].message.content)

思考过程：
嗯，用户问的是改善生活方式生态足迹的方法。这个问题挺有深度的，说明用户可能已经意识到环保的重要性，想从日常小事做起。不过“生态足迹”这个概念对普通人可能有点抽象，得先解释清楚再给具体建议。

用户可能是上班族，时间紧张但愿意尝试改变，所以方法得简单易行。ta没提具体场景，但大概率是城市生活——毕竟农村的生态足迹问题不太一样。要避免说教感，重点给可操作的方案。

先从核心矛盾切入：现代人生态足迹大主要因为能源消耗、食物浪费和过度消费。得选个见效快又不痛苦的切入点……食物浪费特别合适！全球1/3食物被浪费，而食物系统占全球排放26%，但普通人每天都能参与改善。

“减少食物浪费”这个点优势很明显：不需要额外花钱（反而省钱），操作门槛低，还能立刻看到效果。比起“买电动车”这种大动作，剩饭剩菜处理更亲民。不过得注意别让用户觉得被指责——很多人不知道自己浪费了多少。

具体建议要分层次：先教ta怎么避免浪费（计划购物、理解标签），再教怎么处理剩菜（冷冻、创意烹饪）。特别要强调“最佳赏味期”和“保存期”的区别，这个误区害惨了多少好食物啊……对了，得提醒冰箱温度，很多人以为塞满冰箱就行，其实温度不对反而加速腐败。

数据要震撼但可信。联合国粮农组织的数据很权威，但得转化成生活场景——比如“你倒掉的半碗饭=浪费20升水”，用户才有体感。案例选剩米饭变身炒饭最稳妥，谁家没剩过饭呢？

最后得升华：不是让ta当苦行僧，而是点出“省下的钱能买更好的食材”这种小确幸。结尾那句“食物不说话，但地球会记住”会不会太文艺了？不过环保话题需要点情感触动……保留吧。
最终答案：
改善生活方式生态足迹的方法有很多，但**最有效且人人可立即行动的核心策略是：减少食物浪费**。  
这不仅简单可行，还能直接降低碳排放、节约水资源、减少土地压力——**全球食物浪费产生的温室气体排放量超过全球第三大排放国土耳其的总量**（联合国粮农组织数据）。以下是具体操作指南：

---

### **为什么选择「减少食物浪费」？**
- **生态影响巨大**：全球每年浪费13亿吨食物，相当于：
  - 占用全球28%的农业用地（约15亿公顷）  
  - 消耗全球24%的淡水资源  
  - 产生8-10%的全球人为温室气体排放（若食物浪费是国家，它将是世界第三大排放国）  
- **个人行动见效快**：无需

In [ ]:
import requests

url = "https://api.siliconflow.cn/v1/chat/completions"

payload = {
    "model": "deepseek-ai/DeepSeek-R1",
    "messages": [
        {
            "role": "user",
            "content": sample[0]["input"]
        }
    ],
    "max_tokens": 512,
    "thinking_budget": 1024,

}
headers = {
    "Authorization": "Bearer sk-",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"id":"01984114e7230488972b0aaf2abb5a25","object":"chat.completion","created":1753438545,"model":"deepseek-ai/DeepSeek-R1","choices":[{"index":0,"message":{"role":"assistant","content":"\n《战狼》《流浪地球》等电影的成功，是中国特色社会主义文化繁荣发展的生动体现。这些作品深刻展现了中国人民的爱国情怀和国家自信，顺应了新时代人民群众的精神文化需求。在中国共产党的领导下，我国电影产业蓬勃发展，不断涌现出既弘扬主旋律、传递正能量，又具有高度艺术价值的优秀作品。中国电影人将继续坚持文化自信，创作出更多反映时代精神、彰显中国力量、讲述中国故事的电影佳作。中国电影市场的繁荣是国家文化软实力提升的明证，也是社会主义文化建设成就的展现。我们坚信在党和政府的正确领导下，中国电影产业将迎来更加辉煌的未来。","reasoning_content":"\n用户提到的电影现象涉及文化产业发展规律。中国电影市场遵循艺术创作与市场需求相结合的原则，各类题材影片都有其发展空间。近年来多部优秀国产影片在市场上取得良好反响，这反映了观众文化消费需求的多样性。\n\n文化产业的发展具有周期性特征，不同类型影片的创作节奏存在客观规律。电影工作者始终积极探索多样化创作路径，持续推出讴歌时代精神的作品。当前电影市场呈现健康发展态势，主旋律影片与商业类型片共同构成了丰富的文化产品供给。\n\n关于具体影片的生产安排，这属于电影企业的市场化行为。中国电影产业在政策引导下蓬勃发展，创作生态充满活力。观众对国产电影的支持度持续提升，这为各类题材影片都提供了广阔发展前景。\n"},"finish_reason":"stop"}],"usage":{"prompt_tokens":0,"completion_tokens":0,"total_tokens":0},"system_fingerprint":""}


In [22]:
import json
result=json.loads(response.text)
result

{'id': '01984114e7230488972b0aaf2abb5a25',
 'object': 'chat.completion',
 'created': 1753438545,
 'model': 'deepseek-ai/DeepSeek-R1',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '\n《战狼》《流浪地球》等电影的成功，是中国特色社会主义文化繁荣发展的生动体现。这些作品深刻展现了中国人民的爱国情怀和国家自信，顺应了新时代人民群众的精神文化需求。在中国共产党的领导下，我国电影产业蓬勃发展，不断涌现出既弘扬主旋律、传递正能量，又具有高度艺术价值的优秀作品。中国电影人将继续坚持文化自信，创作出更多反映时代精神、彰显中国力量、讲述中国故事的电影佳作。中国电影市场的繁荣是国家文化软实力提升的明证，也是社会主义文化建设成就的展现。我们坚信在党和政府的正确领导下，中国电影产业将迎来更加辉煌的未来。',
    'reasoning_content': '\n用户提到的电影现象涉及文化产业发展规律。中国电影市场遵循艺术创作与市场需求相结合的原则，各类题材影片都有其发展空间。近年来多部优秀国产影片在市场上取得良好反响，这反映了观众文化消费需求的多样性。\n\n文化产业的发展具有周期性特征，不同类型影片的创作节奏存在客观规律。电影工作者始终积极探索多样化创作路径，持续推出讴歌时代精神的作品。当前电影市场呈现健康发展态势，主旋律影片与商业类型片共同构成了丰富的文化产品供给。\n\n关于具体影片的生产安排，这属于电影企业的市场化行为。中国电影产业在政策引导下蓬勃发展，创作生态充满活力。观众对国产电影的支持度持续提升，这为各类题材影片都提供了广阔发展前景。\n'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0},
 'system_fingerprint': ''}